code link: https://www.kaggle.com/code/neerajmohan/fine-tuning-bert-for-text-classification

In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import string
from tabulate import tabulate

import torch
import gc
gc.collect()
torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split

from transformers import BertForSequenceClassification, AdamW,BertTokenizer,get_linear_schedule_with_warmup

### Veri Okuma

In [2]:
df_dataset = pd.read_csv("../dataset/dataset_without_stopwords.csv").drop(columns=["baslik"])

In [6]:
temp_lyrics = [lyrics.replace("\n", "[SEP]").rstrip("[SEP]") for lyrics in df_dataset.loc[:,"sarki_sozu"]]
df_dataset["sarki_sozu"] = temp_lyrics

In [7]:
lyrics = df_dataset.loc[:,"sarki_sozu"].values
labels = df_dataset.loc[:,"label"].values

### BERT Tokenizer

In [8]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased', do_lower_case=True)

In [9]:
print(' Original: ', lyrics[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(lyrics[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(lyrics[0])))

 Original:  çırpınıp şan ovaya çıkınca[SEP]eğlen şan ovada kal acem kızı[SEP]kaş altından bakınca[SEP]can telef ediyor gül acem kızı[SEP]seni seven oğlan neylesin malı[SEP]yumdukça gözünden döker mercanı[SEP]burnu fındık ağzı kahve fincanı[SEP]şeker şerbet bal acem kızı
Tokenized:  ['cı', '##r', '##p', '##ınıp', 'san', 'ova', '##ya', 'cık', '##ınca', '[SEP]', 'eg', '##len', 'san', 'ova', '##da', 'kal', 'ac', '##em', 'kızı', '[SEP]', 'kas', 'altından', 'bakınca', '[SEP]', 'can', 'tel', '##ef', 'ediyor', 'gu', '##l', 'ac', '##em', 'kızı', '[SEP]', 'seni', 'seven', 'og', '##lan', 'ney', '##lesi', '##n', 'malı', '[SEP]', 'yum', '##duk', '##ca', 'goz', '##und', '##en', 'dok', '##er', 'mer', '##can', '##ı', '[SEP]', 'burn', '##u', 'fındık', 'ag', '##zı', 'kahve', 'fincan', '##ı', '[SEP]', 'sek', '##er', 'ser', '##bet', 'bal', 'ac', '##em', 'kızı']
Token IDs:  [17488, 1028, 1054, 14155, 2507, 30766, 2029, 20294, 3754, 3, 5405, 2114, 2507, 30766, 1986, 2235, 3601, 2009, 6595, 3, 3567, 12393, 2

In [10]:
max_len = 0

# For every sentence...
for sent in lyrics:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)
max_len = 512

Token indices sequence length is longer than the specified maximum sequence length for this model (605 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  1619


In [13]:
input_ids = []
attention_masks = []

# For every tweet...
for lyric in lyrics:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        lyric,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

labels = np.array(labels).astype('int64')
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
#print('Original: ', lyrics[0])
#print('Token IDs:', input_ids[0])


tokens = tokenizer.tokenize(tokenizer.decode(input_ids[1]))
token_ids = [i.numpy() for i in input_ids[1]]
attention = [i.numpy() for i in attention_masks[1]]
table = np.array([tokens, token_ids, attention]).T
print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │           2 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ac       │        3601 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##ma     │        1981 │                1 │
├──────────┼─────────────┼──────────────────┤
│ zul      │       10554 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##uf     │        6474 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##lerin  │        2208 │                1 │
├──────────┼─────────────┼──────────────────┤
│ yel      │        8681 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##lere   │        2421 │                1 │
├──────────┼─────────────┼──────────────────┤
│ [SEP]    │           3 │                1 │
├──────────┼─────────────┼──────────────────┤
│ senin    │        3475 │        

Train Validation Split

In [12]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels) # type: ignore

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
#val_size = int(0.2 * len(dataset))
val_size = len(dataset)  - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  960 training samples
  240 validation samples


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [14]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 2

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [15]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 6, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()
model = model.to(device) # type: ignore

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

In [16]:
optimizer = AdamW(model.parameters(), # type: ignore
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

c:\Users\HasanErdemAK\anaconda3\envs\nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Fine Tuning the Model

In [17]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [18]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [19]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [20]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

gc.collect()
torch.cuda.empty_cache()



# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train() # type: ignore
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)         # type: ignore
        loss = output.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # type: ignore
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval() # type: ignore
    # Tracking variables 
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels) # type: ignore
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)

        gc.collect()
        torch.cuda.empty_cache()
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, '../models/bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...

  Average training loss: 1.08
  Training epcoh took: 0:04:15

Running Validation...
  Accuracy: 0.77

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.67
  Training epcoh took: 0:04:15

Running Validation...
  Accuracy: 0.82

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.49
  Training epcoh took: 0:04:17

Running Validation...
  Accuracy: 0.80

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.26
  Training epcoh took: 0:04:16

Running Validation...
  Accuracy: 0.85

Training complete!
Total training took 0:18:46 (h:mm:ss)
